In [1]:
midifile = 'data/chopin-fantaisie.mid'

In [2]:
import time
import copy
import subprocess
from abc import abstractmethod

from midipattern import MidiPattern
from distorter import *

# Midi file parser
import midi

device 0 ('ALSA', 'Midi Through Port-0', 0, 1, 0)
device 1 ('ALSA', 'Midi Through Port-0', 1, 0, 0)
device 2 ('ALSA', 'TiMidity port 0', 0, 1, 0)
device 3 ('ALSA', 'TiMidity port 1', 0, 1, 0)
device 4 ('ALSA', 'TiMidity port 2', 0, 1, 0)
device 5 ('ALSA', 'TiMidity port 3', 0, 1, 0)


In [3]:
MidiPattern.MIDI_DEVICE = 2

Init Pygame and Audio
--------

Midi Pattern
--------

In [4]:
pattern = MidiPattern(midi.read_midifile(midifile))
simple = pattern.simplified(bpm=160)
simple.stamp_time('t0')
midi.write_midifile("generated/simple.mid", simple)

In [5]:
print simple.attributes[0][-40:]

[{'t0': 203.24999999999994}, {'t0': 203.25468749999993}, {'t0': 203.34843749999993}, {'t0': 203.34843749999993}, {'t0': 203.43749999999994}, {'t0': 203.44218749999993}, {'t0': 203.53124999999994}, {'t0': 203.53124999999994}, {'t0': 203.62499999999994}, {'t0': 203.62499999999994}, {'t0': 203.71874999999994}, {'t0': 203.71874999999994}, {'t0': 203.81249999999994}, {'t0': 203.81249999999994}, {'t0': 203.90624999999994}, {'t0': 203.90624999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 203.99999999999994}, {'t0': 204.04687499999994}, {'t0': 204.09374999999994}, {'t0': 204.14062499999994}, {'t0': 204.18749999999994}, {'t0': 204.28124999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.49999999999994}, {'t0': 205.54687499999994}, {'t0': 205.59374999999994}, {'t0': 205.64062499999994}, {'t0': 206.99999999999994}, {'t0': 206.99999999

In [6]:
pattern[0]
pattern.play(180)

Was playing note 5 time 2.66666666667


In [7]:
simple.play()

Was playing note 5 time 3.0


Distorter
--------

In [8]:
distorter = VelocityNoiseDistorter(sigma=20.)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-noise.mid', dist_pattern)
dist_pattern.play(bpm=180)

VelocityNoiseDistorter(sigma=13.99)
Was playing note 24 time 3.55555555556


In [9]:
print dist_pattern.attributes[0][-4:]

[{'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 209.52109374999995, 't0': 209.52109374999995}]


In [10]:
distorter = VelocityWalkDistorter(sigma=0.1)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-walk.mid', dist_pattern)
dist_pattern.play(bpm=180)

VelocityWalkDistorter(sigma=17.44, min=0.39, max=0.43)
Was playing note 20 time 3.33333333333


In [11]:
distorter = ProgramDistorter()
distorter.randomize()
# for some reason GM 1- 3 makes no sound in pygame?
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/program.mid', dist_pattern)
dist_pattern.play(bpm=180)

ProgramDistorter(ticks=0.00)
Was playing note 19 time 2.66666666667


In [12]:
distorter = TempoDistorter(sigma=0, min=0.5, max=2.)
distorter.randomize()
print distorter
print 'time warp', dist_pattern.attributes[0][-4:]
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/tempo.mid', dist_pattern)
dist_pattern.play(bpm=180)

TempoDistorter(sigma=0.48, min=0.72, max=1.49)
time warp [{'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 206.99999999999994, 't0': 206.99999999999994}, {'t': 209.52109374999995, 't0': 209.52109374999995}]
Was playing note 24 time 0.779861111111


In [13]:
distorter = TimeNoiseDistorter()
distorter.randomize()
print distorter
print 'time warp', dist_pattern.attributes[0][-4:]
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/time.mid', dist_pattern)
dist_pattern.play(bpm=180)

TimeNoiseDistorter(sigma=0.01)
time warp [{'t': 193.73072916666104, 't0': 206.99999999999994}, {'t': 193.73151041666102, 't0': 206.99999999999994}, {'t': 193.732291666661, 't0': 206.99999999999994}, {'t': 193.83151041666102, 't0': 209.52109374999995}]
Was playing note 5 time 2.66319444444


Individual Note Times to Global Alignment
-------

In [ ]:
stride = 1.
align = align_frame_to_frame(dist_pattern, stride)
align

In [ ]:
write_align('generated/align.txt', align, stride)
align2, stride2 = read_align('generated/align.txt')
print align2 == align
print int(stride2) == int(stride), stride2, stride

In [ ]:
dist_pattern = random_distort(simple)
align = align_frame_to_frame(dist_pattern, stride=1.)
print align
dist_pattern.play()

Actual Generation
----

In [ ]:
num_samples = 10
stride = 0.1
for i in xrange(num_samples):
    base_name = 'generated/sample-{}'.format(i)
    align_name = '{}.txt'.format(base_name)
    midi_name = '{}.mid'.format(base_name)
    wav_name = '{}.wav'.format(base_name)
    distorted = random_distort(base_name)
    align = align_frame_to_frame(distorted, stride)
    write_align(align_name, align, stride)
    midi.write_midifile(midi_name, distorted)
    # Convert to wav using timidity
    print wav_name
    subprocess.check_call(['timidity', '-Ow', midi_name, '-o', wav_name])
    print 'Done generating {}'.format(base_name)